<a href="https://colab.research.google.com/github/gustavocac/COVID_handson/blob/master/COVID_AI_Dasa_Educa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](https://github.com/gustavocac/COVID_hanson/blob/master/cdc-k0KRNtqcjfw-unsplash.jpg?raw=true)
Photo by CDC on Unsplash


# COVID 2019 

# Hands-on - Deep Learning for COVID Detection on CT 



**Developed by:**
Gustavo C. A. Corradi

Originally Branched and adapted from:

https://colab.research.google.com/github/gustavocac/rsna_handson/blob/master/RSNA_for_Non_Coders_final_version.ipynb


All the process will be demonstrated with Python 3 running on Google Colaboratory. 
Please make sure you have GPU enabled under notebook settings before you proceed.

Validation and Test sets have  normal and  COVID, except for dataset.

For each specific task we will import specific libraries.

After installation the the system will restart. This will generate an error message which is expected. 




In [6]:

import sys, os
!pip3 install keras-vis
!pip3 install imgaug==0.2.5
!pip3 install scipy==1.2.1
!pip3 install graphviz
!pip uninstall matplotlib --yes
!pip install matplotlib==3.1.0
!apt install p7zip-full



     |████████████████████████████████| 563kB 2.7MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-cp36-none-any.whl size=561439 sha256=16e8af245f89c9e09a8f83480e1cee19de018aa1371201bfb6524b22b651906c
  Stored in directory: /root/.cache/pip/wheels/31/48/c8/ca3345e8582a078de94243996e148377ef66fdb845557bae0b
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
     |████████████████████████████████| 24.8MB 1.5MB/s 
ERROR: tensorflow 2.2.0rc3 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.2.1 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
Uninstalling matplotlib-3.2.1:
  Successfully uninstalled matplotlib-3.2.1
     |████████████████████████████████| 13.1MB 327kB/s 
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompat

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Funções para Download no google Drive

In [7]:
!wget https://github.com/UCSD-AI4H/COVID-CT/blob/master/Images-processed/CT_COVID.zip?raw=true
!wget https://github.com/UCSD-AI4H/COVID-CT/blob/master/Images-processed/CT_NonCOVID.zip?raw=true

--2020-04-19 19:31:43--  https://github.com/UCSD-AI4H/COVID-CT/blob/master/Images-processed/CT_COVID.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/UCSD-AI4H/COVID-CT/raw/master/Images-processed/CT_COVID.zip [following]
--2020-04-19 19:31:43--  https://github.com/UCSD-AI4H/COVID-CT/raw/master/Images-processed/CT_COVID.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/UCSD-AI4H/COVID-CT/master/Images-processed/CT_COVID.zip [following]
--2020-04-19 19:31:43--  https://raw.githubusercontent.com/UCSD-AI4H/COVID-CT/master/Images-processed/CT_COVID.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|

In [8]:

!unzip -o CT_NonCOVID.zip?raw=true 
!unzip -o CT_COVID.zip?raw=true



Archive:  CT_NonCOVID.zip?raw=true
  inflating: CT_NonCOVID/0.jpg       
  inflating: CT_NonCOVID/1%0.jpg     
  inflating: CT_NonCOVID/1%1.jpg     
  inflating: CT_NonCOVID/1%2.jpg     
  inflating: CT_NonCOVID/10%0.jpg    
  inflating: CT_NonCOVID/10%1.jpg    
  inflating: CT_NonCOVID/10%2.jpg    
  inflating: CT_NonCOVID/102.png     
  inflating: CT_NonCOVID/1029.png    
  inflating: CT_NonCOVID/103.png     
  inflating: CT_NonCOVID/1030.png    
  inflating: CT_NonCOVID/104.png     
  inflating: CT_NonCOVID/1046.png    
  inflating: CT_NonCOVID/1047.png    
  inflating: CT_NonCOVID/1048.png    
  inflating: CT_NonCOVID/1049.png    
  inflating: CT_NonCOVID/1050.png    
  inflating: CT_NonCOVID/1058.png    
  inflating: CT_NonCOVID/1059.png    
  inflating: CT_NonCOVID/1060.png    
  inflating: CT_NonCOVID/1061.png    
  inflating: CT_NonCOVID/1062.png    
  inflating: CT_NonCOVID/1065.png    
  inflating: CT_NonCOVID/1070.png    
  inflating: CT_NonCOVID/1071.png    
  inflating: CT

Reset for next experiment

Dataset Selection and Preprocessing 

In [9]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
DIR = './CT_COVID'
DIRNC = './CT_NonCOVID'
cov=len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
non_cov= len([name for name in os.listdir(DIRNC) if os.path.isfile(os.path.join(DIRNC, name))])
print("Número de casos:")
print(cov)
print("Número de não casos:")
print(non_cov)
def f(x):
    return x



Número de casos:
349
Número de não casos:
397


In [10]:
import os
print("Escolha o número de cortes positivos no dataset")
COVID=interactive(f, x=widgets.IntSlider(min=0, max=cov, step=1, value=35));

display(COVID)
print("Escolha o número de cortes não positivos no dataset")
N_COVID=interactive(f, x=widgets.IntSlider(min=0, max=non_cov, step=1, value=35));
display(N_COVID)
#print(COVID)

Escolha o número de cortes positivos no dataset


interactive(children=(IntSlider(value=35, description='x', max=349), Output()), _dom_classes=('widget-interact…

Escolha o número de cortes não positivos no dataset


interactive(children=(IntSlider(value=35, description='x', max=397), Output()), _dom_classes=('widget-interact…



```
Separando 60% para treino, 10% para validação e 30% para teste.
```



In [0]:
C=COVID.result
N=N_COVID.result
CTR=round(0.6*C)
CV=round(0.1*C)
CTE=round(0.3*C)
NTR=round(0.6*N)
NV=round(0.1*N)
NTE=round(0.3*N)

In [0]:
import os
import sys
import shutil
import glob

# Get directory name
mydir= "cases/"

try:
    shutil.rmtree(mydir)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))
!mkdir cases
!mkdir cases/temp/
!mkdir cases/temp/COVID
!mkdir cases/temp/nonCOVID
!mkdir cases/Validation/
!mkdir cases/Test/
!mkdir cases/Training/


In [0]:
from shutil import copyfile
from os import listdir
files_C=glob.glob("./CT_COVID/*.*")
files_NC=glob.glob("./CT_NonCOVID/*.*")
x=1
x1=1
x2=1
for f in files_NC:
    if x<=NTR:
      y=str(x)
      shutil.copy(f,'./cases/Training/N'+y+".png")
    x=x+1
    if x>NTR:
      if x1<=NV:
         y=str(x)
         shutil.copy(f,'./cases/Validation/N'+y+".png")
      x=x+1
      x1=x1+1
      if x1>NV:
        if x2<=NTE:
           y=str(x)
           shutil.copy(f,'./cases/Test/N'+y+".png")
           x=x+1
           x1=x1+1
           x2=x2+1
x=1
x1=1
x2=1
for f in files_C:
    if x<=CTR:
      y=str(x)
      shutil.copy(f,'./cases/Training/C'+y+"c.png")
    x=x+1
    if x>CTR:
      if x1<=CV:
         y=str(x)
         shutil.copy(f,'./cases/Validation/C'+y+"c.png")
      x=x+1
      x1=x1+1
      if x1>CV:
        if x2<=NTE:
           y=str(x)
           shutil.copy(f,'./cases/Test/C'+y+"c.png")
           x=x+1
           x1=x1+1
           x2=x2+1




In [49]:
#@title Dataset Selection and Preprocessing {display-mode: "form"}
import os

#Dataset = "397/40" #@param ["397/40", "160/160", "349/349"]
Augmentation = "off" #@param ["off", "on", "custom"]
angle = 359 #@param {type:"slider", min:0, max:359, step:1}
zoom = 1 #@param {type:"slider", min:0, max:1, step:0.1}
width_shift_range = 1 #@param {type:"slider", min:0, max:1, step:0.1}
height_shift_range = 1 #@param {type:"slider", min:0, max:1, step:0.1}
shear_range = 1 #@param {type:"slider", min:0, max:1, step:0.1}
horizontal_flip = True #@param {type:"boolean"}

# Importing libraries for arrays (NumPy), Pre-processing (Keras) and plotting images (Matplotlib)
%tensorflow_version 1.x
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from glob import glob
from IPython.display import clear_output

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# It is important to set a random seed in order to have reproducbility of training results between different users
# SET ALL THE SEEDS
from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
set_random_seed(123)
os.environ['PYTHONHASHSEED'] = '123'
import random
random.seed(123)


TensorFlow 1.x selected.


Using TensorFlow backend.


In [53]:
import os
# Dimensions which our images will be resized for the input. All of them must have the same size
validation_data_dir = './cases/Validation/'
test_data_dir = './cases/Test/'
train_data_dir = './cases/Training/'
nb_train_samples=len([name for name in os.listdir(train_data_dir) if os.path.isfile(os.path.join(train_data_dir, name))])
nb_test_samples= len([name for name in os.listdir(test_data_dir) if os.path.isfile(os.path.join(test_data_dir, name))])
nb_validation_samples= len([name for name in os.listdir(validation_data_dir) if os.path.isfile(os.path.join(validation_data_dir, name))])
print('Treino:')
print(nb_train_samples)
print('Teste:')
print(nb_test_samples)
print('Validação:')
print(nb_validation_samples)


Treino:
342
Teste:
238
Validação:
57


In [0]:



ds_size = {
    "Train_Hematoma": len(glob(train_data_dir + "Hematoma/*")),
    "Train_Normal": len(glob(train_data_dir + "Normal/*")),
    "Val_Hematoma": len(glob(validation_data_dir + "Hematoma/*")),
    "Val_Normal": len(glob(validation_data_dir + "Normal/*")),
    "Test_Hematoma": len(glob(test_data_dir + "Hematoma/*")),
    "Test_Normal": len(glob(test_data_dir + "Normal/*")),
}

# For generator we need to give these two hyperparameters
epochs = 40
batch_size = 10


# This is the augmentation configuration we will use for training
dataaug = Augmentation

if dataaug == "off":
  print("Data Augmentation OFF")
  train_datagen = ImageDataGenerator(
      rescale=1. / 255) # normalization
elif dataaug == "on":
  print("Data Augmentation ON")
  train_datagen = ImageDataGenerator(
      rescale=1. / 255, # normalization
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      rotation_range=20,
      fill_mode="constant",
      horizontal_flip=True)  
else:
  print("Data Augmentation Custom")
  train_datagen = ImageDataGenerator(
      rescale=1. / 255, # normalization
      width_shift_range=width_shift_range,
      height_shift_range=height_shift_range,
      shear_range=shear_range,
      fill_mode="constant",
      zoom_range=zoom, 
      rotation_range=angle,
      horizontal_flip=horizontal_flip)

# This is the augmentation configuration we will use for validation:
val_datagen = ImageDataGenerator(rescale=1. / 255) # normalization

test_datagen = ImageDataGenerator(rescale=1. / 255) # normalization

print("Training set:")
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

print("Validation set:")
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

print("Test set:")
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_test_samples,
    class_mode='binary', shuffle = False)

clear_output()

#Let's plot the class frequencies

print ('\033[1m' + '\r\n          Training Set Distribution')
plt.figure()
plt.bar([0, 1], [ds_size['Train_Hematoma'], ds_size['Train_Normal']], color=['red', 'blue'])
plt.xticks([0, 1], ('Hematoma', 'Normal'))
plt.show()

print ('\033[1m' + '\r\n          Validation Set Distribution')
plt.figure()
plt.bar([0, 1], [ds_size['Val_Hematoma'], ds_size['Val_Normal']], color=['red', 'blue'])
plt.xticks([0, 1], ('Hematoma', 'Normal'))
plt.show()

print ('\033[1m' + '\r\n          Test Set Distribution')
plt.figure()
plt.bar([0, 1], [ds_size['Test_Hematoma'], ds_size['Test_Normal']], color=['red', 'blue'])
plt.xticks([0, 1], ('Hematoma', 'Normal'))
plt.show()

# Let's plot the first 4 generator outputs, defining the positive cases as Label = True and negatives as Label = False 

print ('\033[1m' + '\r\nNow let\'s see some examples\r\n')

x,y = train_generator.next()

labley = y==0
#shape = x.shape
#print (shape)
#for i in range(0, 8):
#  plt.subplot(240 + 1 + i).grid(False)
#  plt.imshow(x[i], cmap=plt.get_cmap('gray'))
#  plt.title("\nLable:{}".format(labley[i]))
#  plt.axis('off')

start_idx = 0
fig, ax = plt.subplots(2,5, figsize=(15,8))
for j in range(0,2): 
  for i in range(0,5):
     ax[j][i].xaxis.set_major_locator(plt.NullLocator())
     ax[j][i].yaxis.set_major_locator(plt.NullLocator())
     ax[j][i].imshow(x[start_idx], cmap='gray')
     ax[j][i].set_title("Index: {} \nLabel: {}".format(start_idx, 'Hematoma' if y[start_idx]==0 else 'Normal'))
     start_idx +=1
plt.show()

# show the plot
plt.show()

print ('\033[1m' + 'Ready for next step!')


<center> <img src="https://github.com/igorafaelms/rsna_handson/blob/master/transfer_learning.png?raw=true"> </center>

In [0]:
#@title Setting Neural Net {display-mode: "form"}

transfer_learning = "VGG16+ImageNet" #@param ["None", "VGG16+ImageNet"]

# This code will be hidden when the notebook is loaded.
# We can improve our results using transfer learning

from keras.applications.vgg16 import VGG16
if transfer_learning=="None":
  base_model = VGG16(weights=None, include_top=False, input_shape=(img_width, img_height, 3))
else:
  base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# Let's edit the last layers of VGG16 to use it in our solution

from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
clear_output()
from keras.models import Model
clear_output()
from keras.callbacks import ModelCheckpoint
clear_output()
from keras import optimizers
clear_output()
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Only for version 2
x = Dense(1024, activation='relu')(x)

# And a logistic layer
predictions = Dense(1, activation='sigmoid')(x)

SIIM_Net= Model(inputs=base_model.input, outputs=predictions)

# We can try using a different optimizer as well

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

SIIM_Net.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])


clear_output()
print ('\033[1m' + 'Ready for next step!')

In [0]:
#@title Training and Validation {display-mode: "form"}
epochs = 20 #@param {type:"slider", min:5, max:80, step:1}
# This code will be hidden when the notebook is loaded.

# Time to train it

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

checkpointer = ModelCheckpoint(filepath='Best_model.hdf5', monitor='val_loss',
                               verbose=1, save_best_only=True)

hist = SIIM_Net.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples // batch_size,
            callbacks=[checkpointer])

#Plotting the loss function

plt.plot(hist.history['loss'], 'b-', label='train loss')
plt.plot(hist.history['val_loss'], 'r-', label='val loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()


plt.plot(hist.history['acc'], 'b-', label='train accuracy')
plt.plot(hist.history['val_acc'], 'r-', label='val accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend()
plt.show()


print('\033[1m' + "Best Validation Accuracy: {:.2f}%".format(hist.history['val_acc'][np.argmin(hist.history['val_loss'])]*100.))
print("  ")

In [0]:
#@title Algorithm Performance (test set) - Score Histogram per Class { display-mode: "form" }

# This code will be hidden when the notebook is loaded.

from keras.models import load_model
from tabulate import tabulate

#Loading the best model

best_model = load_model('Best_model.hdf5')

X, Y = test_generator.next() # Get the X (images) and Y (labels) of the test set

labels_pred = best_model.predict(X) #predict the output from X

labels_test = Y

np.set_printoptions(precision=2)

Y_neg = (1-labels_pred[labels_test == 1])

Y_pos = (1-labels_pred[labels_test == 0])
headers = ["Normal", "Hematoma"]
print(tabulate([[Y_neg,Y_pos]],headers,tablefmt="orgtbl"))

bins = np.linspace(0, 1, 100)

plt.figure()
plt.hist(Y_neg, bins, alpha=0.4, label='Normal', color='b')
plt.hist(Y_pos, bins, alpha=0.4, label='Hematoma', color='r')
plt.legend(loc='upper center')
plt.show()

print ('\033[1m' + 'Ready for next step!')

In [0]:
#@title Algorithm Performance (test set) - 2 x 2 table { display-mode: "form" }
threshold = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
threshold = 1-threshold
# This code will be hidden when the notebook is loaded.

from sklearn import metrics

from keras.models import load_model

#Loading the best model

best_model = load_model('Best_model.hdf5')


# Defining a function to plot a confusion matrix.

# from https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = '2x2 table'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    #else:
        #print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Ground Truth',
           xlabel='Predicted')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[1]):
        for j in range(cm.shape[0]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

X, Y = test_generator.next() # Get the X (images) and Y (labels) of the test set

labels_pred = best_model.predict(X) #predict the output from X

#labels_pred = labels_pred > labels_pred.mean() #predictions greater than mean are set to 1, those lesser than or equal to mean are set to 0.

labels_pred = labels_pred >= threshold

labels_test = Y

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(labels_test, labels_pred.astype('int'), classes=['Hematoma','Normal'], normalize=False,
                      title='Confusion Matrix (Counts)')

accuracy = metrics.accuracy_score(labels_test, labels_pred)

print("Accuracy: {:.2f}%".format(accuracy*100.))

#print ('\033[1m' + 'Ready for next step!')

In [0]:
#@title Algorithm Performance (test set) - ROC Curve {display-mode: "form"}

threshold = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
threshold = 1-threshold

#Plotting the ROC curve with the AUC

labels_pred = best_model.predict(X) # predict again to get the original sigmoid output [0,1]

from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(labels_test, labels_pred)
roc_auc = metrics.auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')

plt.plot(fpr[np.argmin(np.abs(thresholds-threshold))],tpr[np.argmin(np.abs(thresholds-threshold))], 'o', color='red')

plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

#labels_pred = labels_pred > labels_pred.mean() #predictions greater than mean are set to 1, those lesser than or equal to mean are set to 0.

labels_pred = labels_pred >= threshold

f1_score = metrics.f1_score(labels_test, labels_pred, labels=None, pos_label=0, average='binary', sample_weight=None)

accuracy = metrics.accuracy_score(labels_test, labels_pred)

print("Accuracy: {:.2f}%".format(accuracy*100.))

print("F1 Score: {:.4f}".format(f1_score))

print ('\033[1m' + 'Ready for next step!')

In [0]:
#@title Test Evaluation - Prediction on sampled images {display-mode: "form"}
threshold = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
threshold = 1-threshold

# Finally, we can use the test set for predictions

import warnings
warnings.filterwarnings('ignore')

test_data_dir = '/Cases/All cases/Test/' # location of test dataset

test_datagen = ImageDataGenerator(
         rescale=1./255)       # normalize pixel values to [0,1]

# Preparing test set images for prediction

itr = test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=40,
    shuffle='False',
    class_mode='binary')
batch_x, batch_y = itr.next()

print('Test Accuracy: {:.2f}%'.format(100.*best_model.evaluate(batch_x, batch_y, verbose=0)[1]))



from random import randrange

#prediction1 = np.round(best_model.predict(batch_x, verbose=1))==0
prediction = best_model.predict(batch_x, verbose=1)

start_idx = randrange(batch_x.shape[0]-10) 
fig, ax = plt.subplots(2,5, figsize=(15,8))
for j in range(0,2): 
  for i in range(0,5):
     ax[j][i].xaxis.set_major_locator(plt.NullLocator())
     ax[j][i].yaxis.set_major_locator(plt.NullLocator())
     ax[j][i].imshow(batch_x[start_idx], cmap='gray')
     ax[j][i].set_title("Index: {} \nPrediction: {} \nValue: {:.2f}".format(start_idx, 'Hematoma' if prediction[start_idx]<threshold else 'Normal', 1-prediction[start_idx][0]))
     start_idx +=1
plt.show()

print ('\033[1m' + 'Ready for next step!')

In [0]:
#@title Visualization - Saliency Maps {display-mode: "form"}

import warnings
warnings.filterwarnings('ignore')

# Importing visualization tools

from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.layers import Input
from keras import activations
from keras.models import load_model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import initializers
from keras.models import Sequential, Model
from vis.visualization import visualize_activation,visualize_saliency,overlay,visualize_cam
from vis.utils import utils

import matplotlib.pyplot as plt
from keras.applications import imagenet_utils
import numpy as np

layer_idx = utils.find_layer_idx(best_model, 'block5_conv3')
print("Remove Activation from Last Layer")
# Swap softmax with linear
best_model.layers[layer_idx].activation = activations.linear
print("Done. Now Applying changes to the model ...")
activation2_model = utils.apply_modifications(best_model)

#print(activation_model.summary())
#im_files=["/All cases/Test/Hematoma/Test hematoma (1).png","/All cases/Test/Normal/Test_normal (1).png"]
import os
cwd = os.getcwd()
import os

dir_name='/Cases/All cases/Test/'
im_files = test_generator.filenames
for im_file in im_files[3:6]:
    img1 = image.load_img(dir_name + im_file,target_size=(150,150))
    img1 = image.img_to_array(img1)
    img1 = np.expand_dims(img1, axis=0)
    img1 = preprocess_input(img1)
    layer_idx = utils.find_layer_idx(activation2_model, 'block5_conv3')
    heatmap = visualize_cam(activation2_model, layer_idx, filter_indices=range(activation2_model.layers[layer_idx].filters), seed_input=img1[0,:,:,:])
    img_init=utils.load_img(dir_name + im_file,target_size=(150,150))
    img_init = img_init[:,:,:3]
    plt.figure(figsize=(10,10))
    ax1 = plt.subplot(1,3,1)
    ax1.grid(False)
    plt.imshow(img_init, cmap='gray')
    ax2 = plt.subplot(1,3,2)
    ax2.grid(False)
    plt.imshow(heatmap)
    ax3 = plt.subplot(1,3,3)
    ax3.grid(False)
    plt.imshow(overlay(img_init, heatmap))
    plt.show()
    
#print ('\n' + '\033[1m' + 'Congratulations, you have completed the assignment!')

#from IPython.display import HTML
#HTML('<img src="https://media.giphy.com/media/cub3pntkz8muQ/giphy.gif">')